Dado que el entrenamiento de redes neuronales es una tarea  muy costosa, **se recomienda ejecutar el notebooks en [Google Colab](https://colab.research.google.com)**, por supuesto también se puede ejecutar en local.

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

<a name='actividad_1'></a>
# 1: Redes Densas

Para este proyecto he utilizado el [wine quality dataset](https://archive.ics.uci.edu/ml/datasets/wine+quality). Con el que trataremos de predecir la calidad del vino.

La calidad del vino puede tomar valores decimales (por ejemplo 7.25), independientemente de que en el dataset de entrenamiento sean números enteros. Por lo tanto, el problema es una `regresión`.

In [ ]:
# Descargar los datos con pandas
df_red = pd.read_csv('winequality-red.csv', sep=';')
df_white = pd.read_csv('winequality-white.csv', sep=';')

df = pd.concat([df_red, df_white])

df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [ ]:
feature_names = [
    'fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar', 'chlorides',
    'free sulfur dioxide', 'total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol'
]

# separar features y target
y = df.pop('quality').values
X = df.copy().values

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

print('x_train, y_train shapes:', x_train.shape, y_train.shape)
print('x_test, y_test shapes:', x_test.shape, y_test.shape)
print('Some qualities: ', y_train[:5])

x_train, y_train shapes: (4872, 11) (4872,)
x_test, y_test shapes: (1625, 11) (1625,)
Some qualities:  [6 7 8 5 6]


In [ ]:
## Normalizo las features.

# Creo una instancia del escalador StandardScaler.
# Este escalador transforma los datos para que tengan media 0 y desviación estándar 1.
scaler = StandardScaler()

# Ajusto el escalador a los datos de entrenamiento y transforma esos datos.
# Esto me asegura que el modelo vea características con la misma escala.
x_train = scaler.fit_transform(x_train)

# Uso el mismo escalador (ya ajustado con los datos de entrenamiento) para transformar los datos de test.
x_test = scaler.transform(x_test)

<a name='1.1'></a>
## Creo un modelo secuencial que contenga 4 capas ocultas(hidden layers), con más de 60 neuronas  por capa, sin regularización y obtenga los resultados.

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam

# Fijo la semilla para NumPy, para asegurar resultados reproducibles.
np.random.seed(42)

# Fijo la semilla para TensorFlow, para asegurar que los resultados del entrenamiento sean consistentes.
tf.random.set_seed(42)

# Defino el modelo secuencial con una capa de entrada explícita.
# Con ello, mejoro la legibilidad y evito advertencias sobre input_shape.
model = Sequential([
    Input(shape=(x_train.shape[1],)),  # Capa de entrada, que se adapta al número de características de los datos.

    # Creo la primera capa oculta, con 64 neuronas y activación ReLU.
    # Uso la función da activación ReLU, ya que introduce no linealidades, es eficiente computacionalmente
    # y ayuda a mitigar el problema del gradiente desvanecido.
    Dense(64, activation='relu'),

    # Creo la segunda capa oculta, idéntica a la primera.
    Dense(64, activation='relu'),

    # Creo la tercera capa oculta, con las mismas características.
    Dense(64, activation='relu'),

    # Creo la cuarta capa oculta, con las mismas características.
    Dense(64, activation='relu'),

    # Creo la capa de salida, con una única neurona y sin activación.
    # Esto es adecuado para este problema de regresión, donde la salida es un valor continuo.
    Dense(1)
])

In [ ]:
# Compilación del modelo
# Uso el optimizador Adam, que combina lo mejor de RMSprop y momentum, y se adapta bien a la mayoría de los problemas.
# Establezco una tasa de aprendizaje pequeña (0.001) para que el modelo aprenda de forma más estable.
# Como es un problema de regresión, uso 'mse' (error cuadrático medio) como función de pérdida.
# También incluyo 'mse' como métrica para monitorizar el rendimiento durante el entrenamiento y evaluación.

model.compile(
    optimizer=Adam(learning_rate=0.001),  # Optimizador Adam con tasa de aprendizaje 0.001
    loss='mse',                           # Función de pérdida: mean squared error (regresión)
    metrics=['mse']                       # Métrica: también el MSE, para visualizar el error
)

In [ ]:
# No modifico el código
# Se entrena el modelo con los datos de entrenamiento
# Se usan 200 épocas, lo que indica que el modelo verá el conjunto completo de entrenamiento 200 veces.
# batch_size=32 significa que el gradiente se actualiza cada 32 muestras.
# validation_split=0.2 reserva el 20% del conjunto de entrenamiento para validación.
# verbose=1 muestra una barra de progreso por cada época.
model.fit(x_train,
          y_train,
          epochs=200,
          batch_size=32,
          validation_split=0.2,
          verbose=1)


Epoch 1/200
122/122 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - loss: 14.6076 - mse: 14.6076 - val_loss: 1.7840 - val_mse: 1.7840
Epoch 2/200
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.3199 - mse: 1.3199 - val_loss: 1.0235 - val_mse: 1.0235
Epoch 3/200
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.8264 - mse: 0.8264 - val_loss: 0.6824 - val_mse: 0.6824
Epoch 4/200
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.6067 - mse: 0.6067 - val_loss: 0.5796 - val_mse: 0.5796
Epoch 5/200
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.5271 - mse: 0.5271 - val_loss: 0.5400 - val_mse: 0.5400
Epoch 6/200
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.4931 - mse: 0.4931 - val_loss: 0.5257 - val_mse: 0.5257
Epoch 7/200
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.4729 - mse: 0.4729 - val_loss: 0.5171 - val_mse: 0.5171
Epoch 8/200
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.4598 - mse: 0.4598 - val_loss: 0.5110 - val_mse: 0.5110
Epoch 9/200
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 3

In [ ]:
# No modifico el código.
# Se evalúa el modelo utilizando el conjunto de test (datos no vistos durante el entrenamiento).
# Esto permite obtener una estimación objetiva del rendimiento del modelo en datos nuevos.
# verbose=1 muestra el progreso del proceso de evaluación.
results = model.evaluate(x_test, y_test, verbose=1)

# Se imprime el valor de la pérdida (loss) en el conjunto de test.
# En este caso, la métrica corresponde al error cuadrático medio (MSE).
print('Test Loss: {}'.format(results))

51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.7064 - mse: 0.7064
Test Loss: [0.7462330460548401, 0.7462330460548401]


## Explicación de lo realizado

En esta primera cuestión construí un modelo secuencial con cuatro capas ocultas, cada una con 64 neuronas y la función de activación `ReLU`, tal como se especificaba en el enunciado. No añadí ninguna técnica de regularización, ya que el objetivo era trabajar con una arquitectura base sin restricciones adicionales.

La capa de salida contiene una única neurona sin función de activación, lo cual es apropiado para problemas de regresión, donde se espera que el modelo devuelva un valor continuo.

Compilé el modelo utilizando el optimizador `Adam` con una tasa de aprendizaje de 0.001, y como función de pérdida elegí el error cuadrático medio (`mse`), que es la más habitual y adecuada para tareas de regresión.

Entrené la red durante 200 épocas, con un 20% de los datos de entrenamiento reservado para validación (`validation_split=0.2`). Esto me permitió observar cómo evolucionaba la pérdida en los datos de validación y verificar que el modelo aprendía correctamente.

Finalmente, evalué el modelo sobre el conjunto de test y se imprimió el `Test Loss`, tal como solicitaba la consigna.

**Nota:** Aunque fijé la semilla de aleatoriedad con `np.random.seed(42)` y `tf.random.set_seed(42)`, es posible que los resultados varíen ligeramente entre ejecuciones, sobre todo si se usa GPU. Esto se debe a que algunas operaciones de TensorFlow no son completamente deterministas en entornos paralelos.


<a name='1.2'></a>
## Cuestión 2: Utilice el mismo modelo de la cuestión anterior pero añadiendo al menos dos técnicas distinas de regularización. No es necesario reducir el test loss.

Ejemplos de regularización: [Prevent_Overfitting.ipynb](https://github.com/ezponda/intro_deep_learning/blob/main/class/Fundamentals/Prevent_Overfitting.ipynb)

In [ ]:
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Input
from tensorflow.keras.regularizers import l2

# Fijo la semilla para NumPy, para asegurar resultados reproducibles.
np.random.seed(42)

# Fijo la semilla para TensorFlow, para asegurar que los resultados del entrenamiento sean consistentes.
tf.random.set_seed(42)

# Defino el modelo secuencial con regularización L2 y una capa de entrada explícita.
model = Sequential([
    Input(shape=(x_train.shape[1],)),  # Capa de entrada que se adapta al número de características del dataset.

    # Primera capa oculta
    # Aplico regularización L2 para penalizar pesos grandes (técnica 1)
    Dense(64, activation='relu', kernel_regularizer=l2(0.001)),

    # Aplico Batch Normalization para estabilizar el entrenamiento (técnica 2)
    BatchNormalization(),

    # Aplico Dropout para evitar sobreajuste (25% de neuronas desactivadas aleatoriamente) (técnica 3)
    Dropout(0.25),

    # Segunda capa oculta con las tres técnicas también
    Dense(64, activation='relu', kernel_regularizer=l2(0.001)),  # L2
    BatchNormalization(),                                        # BatchNormalization
    Dropout(0.25),                                               # Dropout

    # Tercera capa oculta
    Dense(64, activation='relu', kernel_regularizer=l2(0.001)),  # L2
    BatchNormalization(),                                        # BatchNormalization
    # Aquí no uso Dropout, pero sigue habiendo regularización L2 + BatchNorm

    # Cuarta capa oculta
    Dense(64, activation='relu', kernel_regularizer=l2(0.001)),  # Solo regularización L2

    # Capa de salida: una sola neurona sin activación (regresión)
    Dense(1)
])

In [ ]:
# Compilación del modelo.

# Uso el optimizador Adam con una tasa de aprendizaje pequeña (0.001),
# que suele funcionar bien en muchos problemas de deep learning.
optimizer = Adam(learning_rate=0.001)

# Compilo el modelo usando como función de pérdida el error cuadrático medio (MSE),
# ya que se trata de un problema de regresión.
# También añado el MSE como métrica para monitorizar durante el entrenamiento.
model.compile(
    optimizer=optimizer,  # Optimizador Adam
    loss='mse',           # Función de pérdida: error cuadrático medio
    metrics=['mse']       # Métrica usada para evaluar el desempeño del modelo
)

In [ ]:
# Defino el tamaño del batch (lote) que se usará en cada paso de entrenamiento.
# Uso un batch_size de 32, ya que es un valor común, que ofrece un buen equilibrio entre velocidad de entrenamiento y estabilidad.
batch_size = 32

In [ ]:
# No modifico el código.
# Se entrena el modelo usando los datos de entrenamiento (x_train, y_train).
# El entrenamiento se realiza durante 200 épocas.
# batch_size define cuántas muestras se procesan antes de actualizar los pesos.
# validation_split=0.2 separa el 20% de los datos de entrenamiento para validación.
# verbose=1 muestra el progreso del entrenamiento en pantalla.

model.fit(x_train,
          y_train,
          epochs=200,
          batch_size=batch_size,
          validation_split=0.2,
          verbose=1)

Epoch 1/200
122/122 ━━━━━━━━━━━━━━━━━━━━ 7s 22ms/step - loss: 27.0415 - mse: 26.8322 - val_loss: 4.6031 - val_mse: 4.3877
Epoch 2/200
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.5268 - mse: 1.3113 - val_loss: 0.9640 - val_mse: 0.7487
Epoch 3/200
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.0872 - mse: 0.8721 - val_loss: 0.8206 - val_mse: 0.6062
Epoch 4/200
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.9870 - mse: 0.7730 - val_loss: 0.8316 - val_mse: 0.6185
Epoch 5/200
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.9511 - mse: 0.7384 - val_loss: 0.8030 - val_mse: 0.5913
Epoch 6/200
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.9167 - mse: 0.7055 - val_loss: 0.7859 - val_mse: 0.5759
Epoch 7/200
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.9143 - mse: 0.7047 - val_loss: 0.7741 - val_mse: 0.5658
Epoch 8/200
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.8571 - mse: 0.6493 - val_loss: 0.7495 - val_mse: 0.5429
Epoch 9/200
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 3

In [ ]:
# No modifico el código.
# Se evalúa el modelo con el conjunto de test (x_test, y_test),
# para obtener el rendimiento final sobre datos no vistos durante el entrenamiento.

results = model.evaluate(x_test, y_test, verbose=1)

# Se Imprime la pérdida obtenida en el conjunto de test.
# Lo que permite comprobar, el error del modelo una vez entrenado.
print('Test Loss: {}'.format(results))

51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.5271 - mse: 0.4637
Test Loss: [0.5438356399536133, 0.4804173409938812]


## Explicación de lo realizado

En esta segunda cuestión utilicé la misma arquitectura del modelo que en la pregunta anterior: una red secuencial con 4 capas ocultas, cada una con 64 neuronas y activación ReLU, y una capa de salida con una sola neurona (sin activación), adecuada para regresión.

Para cumplir con el objetivo de aplicar al menos **dos técnicas distintas de regularización**, incorporé las siguientes:

- **Regularización L2** (`kernel_regularizer=l2(0.001)`) en todas las capas ocultas, para penalizar los pesos grandes y evitar que el modelo se sobreentrene.
- **Dropout**, con una tasa del 25% después de las dos primeras capas ocultas, que desactiva aleatoriamente neuronas durante el entrenamiento y mejora la generalización.
- **Batch Normalization**, después de cada capa oculta (excepto la última), para estabilizar y acelerar el aprendizaje ajustando la distribución de activaciones intermedias.

Además, fijé una semilla con `np.random.seed(42)` y `tf.random.set_seed(42)` para mejorar la reproducibilidad.

Compilé el modelo usando el optimizador **Adam** (por su buen rendimiento y ajuste automático de la tasa de aprendizaje) y la función de pérdida **MSE** (adecuada para tareas de regresión). Entrené la red durante 200 épocas con `batch_size = 32` y un `validation_split = 0.2`.

Finalmente, evalué el modelo sobre el conjunto de test, tal como se indicaba en el enunciado.


<a name='1.3'></a>
## Cuestión 3: Utilice el mismo modelo de la cuestión anterior pero añadiendo un callback de early stopping. No es necesario reducir el test loss.

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

# Fijo la semilla para NumPy, para asegurar resultados reproducibles.
np.random.seed(42)

# Fijo la semilla para TensorFlow, para asegurar que los resultados del entrenamiento sean consistentes.
tf.random.set_seed(42)

# Defino el modelo secuencial con regularización L2, normalización y una capa de entrada explícita.
model = Sequential([
    Input(shape=(x_train.shape[1],)),  # Capa de entrada que se adapta al número de características del dataset

    # Primera capa oculta con 64 neuronas, activación ReLU y regularización L2
    Dense(64, activation='relu', kernel_regularizer=l2(0.001)),
    BatchNormalization(),  # Normaliza las salidas para mejorar la estabilidad del entrenamiento
    Dropout(0.25),         # Apaga aleatoriamente el 25% de las neuronas (regularización)

    # Segunda capa oculta con las mismas características
    Dense(64, activation='relu', kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    Dropout(0.25),

    # Tercera capa oculta con normalización pero sin dropout
    Dense(64, activation='relu', kernel_regularizer=l2(0.001)),
    BatchNormalization(),

    # Cuarta capa oculta solo con regularización L2
    Dense(64, activation='relu', kernel_regularizer=l2(0.001)),

    # Capa de salida con una única neurona (sin activación, porque es un problema de regresión)
    Dense(1)
])

In [ ]:
# Compilación del modelo.
# Utilizo el optimizador Adam con una tasa de aprendizaje de 0.001, adecuado para la mayoría de tareas de regresión.
# Empleo la función de pérdida, que es el error cuadrático medio (MSE), ideal para problemas donde se predicen valores continuos.
# También incluyo el MSE como métrica para monitorizar el rendimiento durante el entrenamiento.
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='mse',
    metrics=['mse']
)

In [ ]:
## defino el early stopping callback.
# Uso EarlyStopping, para evitar entrenar más de lo necesario si no mejora la validación.
early_stop = EarlyStopping(
    monitor='val_loss',         # miro la pérdida en validación.
    patience=10,                # espero 10 épocas sin mejora.
    restore_best_weights=True   # recupero los mejores pesos.
)

model.fit(x_train,
          y_train,
          epochs=200,
          batch_size=32,
          validation_split=0.2,
          verbose=1,
          callbacks=[early_stop])  # aplico el callback de early stopping.

Epoch 1/200
122/122 ━━━━━━━━━━━━━━━━━━━━ 7s 22ms/step - loss: 10.8430 - mse: 10.6244 - val_loss: 3.7275 - val_mse: 3.5035
Epoch 2/200
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.2122 - mse: 0.9884 - val_loss: 1.0323 - val_mse: 0.8093
Epoch 3/200
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.0476 - mse: 0.8250 - val_loss: 0.9419 - val_mse: 0.7204
Epoch 4/200
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.9867 - mse: 0.7657 - val_loss: 0.8452 - val_mse: 0.6256
Epoch 5/200
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.9245 - mse: 0.7053 - val_loss: 0.8308 - val_mse: 0.6131
Epoch 6/200
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.8839 - mse: 0.6668 - val_loss: 0.8218 - val_mse: 0.6062
Epoch 7/200
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.8863 - mse: 0.6712 - val_loss: 0.8259 - val_mse: 0.6125
Epoch 8/200
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.8708 - mse: 0.6579 - val_loss: 0.7975 - val_mse: 0.5864
Epoch 9/200
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 3

In [ ]:
# No modifico el código.
# Se evalúa el modelo entrenado sobre el conjunto de test.
# Esto calcula la pérdida (loss) y las métricas especificadas durante la compilación (en este caso, el MSE).
results = model.evaluate(x_test, y_test, verbose=1)

# Se imprime el resultado de la evaluación (pérdida y métrica) en formato legible.
print('Test Loss: {}'.format(results))

51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.5829 - mse: 0.4910
Test Loss: [0.6117718815803528, 0.5198562741279602]


## Explicación de lo realizado

En esta cuestión partí del mismo modelo usado en la Cuestión 2, que ya incluía diversas técnicas de regularización: regularización **L2**, **Dropout** y **BatchNormalization**, todas aplicadas para reducir el riesgo de sobreajuste en una red profunda.

Además, introduje una novedad importante: el uso del **callback de EarlyStopping**. Esta técnica permite detener el entrenamiento de forma anticipada cuando la pérdida en el conjunto de validación (`val_loss`) deja de mejorar. Esto ayuda a evitar el sobreentrenamiento y a reducir el tiempo de entrenamiento innecesario.

El callback se configuró con:
- `monitor='val_loss'`: se observa la pérdida en validación,
- `patience=10`: se permite un margen de 10 épocas sin mejora antes de detener el entrenamiento,
- `restore_best_weights=True`: se restauran automáticamente los mejores pesos alcanzados.

Como en las cuestiones anteriores:
- Fijé una **semilla para NumPy (`np.random.seed(42)`)** y otra para **TensorFlow (`tf.random.set_seed(42)`)**. Esto busca asegurar resultados más consistentes y reproducibles entre ejecuciones.
- Utilicé el optimizador **Adam** con tasa de aprendizaje 0.001.
- La función de pérdida fue el **error cuadrático medio (`mse`)**, adecuada para problemas de regresión.
- Entrené durante un máximo de 200 épocas, aunque el EarlyStopping detuvo el proceso antes de tiempo al no detectarse mejoras.

Finalmente, evalué el modelo con el conjunto de test, imprimiendo la métrica final (`Test Loss`) tal como se pedía en el enunciado.